# SLiCAP Circuit Analysis in Jupyter

This notebook demonstrates how to use SLiCAP for circuit analysis in Jupyter with inline results and visualizations.

## Setup

First, import SLiCAP and configure for Jupyter notebook display.

In [ ]:
# Import SLiCAP
from SLiCAP import *
import numpy as np
import matplotlib.pyplot as plt

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

# Initialize project
initProject("Jupyter Examples")

print("✓ SLiCAP initialized successfully")
print(f"Version: {ini.version}")

## Example 1: Voltage Divider Analysis

Let's analyze a simple voltage divider circuit.

In [ ]:
# Load the voltage divider circuit
cir_vdiv = makeCircuit('voltage_divider.cir')

# Display circuit information
print("Circuit Title:", cir_vdiv.title)
print("Nodes:", cir_vdiv.nodes)
print("Elements:", list(cir_vdiv.elements.keys()))

In [ ]:
# Define circuit parameters
cir_vdiv.defPar('R1', 1000)   # 1kΩ
cir_vdiv.defPar('R2', 2000)   # 2kΩ
cir_vdiv.defPar('V_in', 10)   # 10V

print("Circuit Parameters:")
print(f"  R1 = 1kΩ")
print(f"  R2 = 2kΩ")
print(f"  V_in = 10V")

In [ ]:
# Perform Laplace analysis
result_vdiv = doLaplace(cir_vdiv, source='V1', detector='V_out')

print("Transfer Function:")
display(result_vdiv.laplace)

print("\nNumerator:")
display(result_vdiv.numer)

print("\nDenominator:")
display(result_vdiv.denom)

In [ ]:
# Calculate output voltage
H = result_vdiv.laplace
dc_gain = H.subs([('R1', 1000), ('R2', 2000), ('s', 0)])
v_out = 10 * float(dc_gain)

print(f"DC Gain: {dc_gain}")
print(f"Output Voltage: {v_out:.4f} V")

# Verify with formula
v_out_formula = 10 * 2000 / (1000 + 2000)
print(f"\nVerification: {v_out_formula:.4f} V")
print(f"Match: {abs(v_out - v_out_formula) < 0.001}")

## Example 2: RC Low Pass Filter

Now let's analyze an RC low pass filter with frequency response.

In [ ]:
# Load RC filter circuit
cir_rc = makeCircuit('rc_lowpass.cir')

# Define parameters
R_val = 1000
C_val = 100e-9
cir_rc.defPar('R', R_val)
cir_rc.defPar('C', C_val)
cir_rc.defPar('V_in', 1)

# Calculate cutoff frequency
fc = 1 / (2 * np.pi * R_val * C_val)

print("RC Low Pass Filter")
print("="*50)
print(f"R = {R_val/1000} kΩ")
print(f"C = {C_val*1e9} nF")
print(f"Cutoff frequency fc = {fc:.2f} Hz")
print(f"ωc = {2*np.pi*fc:.2f} rad/s")

In [ ]:
# Get transfer function
result_rc = doLaplace(cir_rc, source='V1', detector='V_out')

print("Transfer Function H(s):")
display(result_rc.laplace)

from sympy import simplify
H_simplified = simplify(result_rc.laplace)
print("\nSimplified:")
display(H_simplified)

### Frequency Response Analysis

In [ ]:
# Evaluate transfer function at different frequencies
frequencies = np.logspace(0, 5, 200)  # 1 Hz to 100 kHz

# Calculate magnitude and phase
H = result_rc.laplace
magnitude = []
phase = []

for f in frequencies:
    omega = 2 * np.pi * f
    H_val = H.subs([('R', R_val), ('C', C_val), ('s', 1j*omega)])
    H_complex = complex(H_val)
    magnitude.append(abs(H_complex))
    phase.append(np.angle(H_complex, deg=True))

magnitude = np.array(magnitude)
phase = np.array(phase)
magnitude_dB = 20 * np.log10(magnitude + 1e-10)

print(f"Frequency response calculated for {len(frequencies)} points")
print(f"Range: {frequencies[0]:.1f} Hz to {frequencies[-1]:.0f} Hz")

In [ ]:
# Plot Bode magnitude
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.semilogx(frequencies, magnitude_dB, 'b-', linewidth=2)
plt.axhline(-3, color='r', linestyle='--', alpha=0.7, label='-3 dB line')
plt.axvline(fc, color='g', linestyle='--', alpha=0.7, label=f'fc = {fc:.1f} Hz')
plt.grid(True, which='both', alpha=0.3)
plt.xlabel('Frequency (Hz)', fontsize=12)
plt.ylabel('Magnitude (dB)', fontsize=12)
plt.title('Bode Plot - Magnitude', fontsize=14, fontweight='bold')
plt.legend()
plt.ylim(-40, 5)

# Plot Bode phase
plt.subplot(1, 2, 2)
plt.semilogx(frequencies, phase, 'b-', linewidth=2)
plt.axhline(-45, color='r', linestyle='--', alpha=0.7, label='-45° line')
plt.axvline(fc, color='g', linestyle='--', alpha=0.7, label=f'fc = {fc:.1f} Hz')
plt.grid(True, which='both', alpha=0.3)
plt.xlabel('Frequency (Hz)', fontsize=12)
plt.ylabel('Phase (degrees)', fontsize=12)
plt.title('Bode Plot - Phase', fontsize=14, fontweight='bold')
plt.legend()
plt.ylim(-95, 5)

plt.tight_layout()
plt.show()

print("✓ Bode plots displayed")

In [ ]:
# Find response at key frequencies
test_freqs = [10, 100, fc, 1000, 10000]

print("\nFrequency Response at Key Points:")
print("="*70)
print(f"{'Frequency':<15} {'|H|':<12} {'|H| (dB)':<12} {'Phase (°)':<12}")
print("-"*70)

for f in test_freqs:
    omega = 2 * np.pi * f
    H_val = H.subs([('R', R_val), ('C', C_val), ('s', 1j*omega)])
    H_complex = complex(H_val)
    mag = abs(H_complex)
    mag_db = 20 * np.log10(mag) if mag > 0 else -np.inf
    ph = np.angle(H_complex, deg=True)
    
    freq_str = f"{f:.1f} Hz" if f != fc else f"{f:.1f} Hz (fc)"
    print(f"{freq_str:<15} {mag:<12.4f} {mag_db:<12.2f} {ph:<12.2f}")

### Pole-Zero Analysis

In [ ]:
# Perform pole-zero analysis
result_pz = doPZ(cir_rc, source='V1', detector='V_out')

print("Pole-Zero Analysis")
print("="*50)

print("\nPoles:")
if hasattr(result_pz, 'poles') and result_pz.poles:
    poles = result_pz.poles if isinstance(result_pz.poles, list) else [result_pz.poles]
    for i, pole in enumerate(poles):
        pole_val = pole.subs([('R', R_val), ('C', C_val)])
        print(f"  p{i+1} = {pole}")
        print(f"     = {complex(pole_val)}")
        print(f"     = {float(pole_val.evalf()):.2f} rad/s")
else:
    print("  No poles found (or already displayed)")

print("\nZeros:")
if hasattr(result_pz, 'zeros') and result_pz.zeros:
    print(f"  {result_pz.zeros}")
else:
    print("  None (no zeros in finite s-plane)")

## Example 3: Display Circuit Schematic

We can also display circuit schematics inline in Jupyter.

In [ ]:
from IPython.display import Image, display
import os

# Display voltage divider schematic
if os.path.exists('img/voltage_divider_schematic.png'):
    print("Voltage Divider Circuit:")
    display(Image('img/voltage_divider_schematic.png', width=400))
else:
    print("Schematic not found. Run create_schematics.py first.")

In [ ]:
# Display RC filter schematic
if os.path.exists('img/rc_lowpass_schematic.png'):
    print("RC Low Pass Filter Circuit:")
    display(Image('img/rc_lowpass_schematic.png', width=400))
else:
    print("Schematic not found. Run create_schematics.py first.")

## Summary

This notebook demonstrated:

1. **Setting up SLiCAP** in Jupyter notebooks
2. **DC Analysis** with voltage divider
3. **AC Analysis** with RC filter
4. **Frequency Response** plotting with matplotlib
5. **Pole-Zero Analysis** for system characterization
6. **Inline Schematics** for circuit visualization

### Key Benefits of Jupyter + SLiCAP

- **Interactive Analysis**: Run cells individually, experiment with parameters
- **Inline Plots**: See results immediately with matplotlib
- **LaTeX Rendering**: Beautiful equation display with sympy
- **Documentation**: Combine code, results, and explanations
- **Reproducibility**: Share complete analysis workflow

### Next Steps

- Try changing circuit parameters and re-running cells
- Add more complex circuits
- Create parametric studies
- Generate custom plots and visualizations